Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## Introduction to Azure Machine Learning service: Run experiment

In this example, you'll learn how to use Azure Machine Learning for experimentation.

First, let's load your workspace and create an experiment. When you run *ws = Workspace.from_config()* below, you will be prompted to log in to your Azure subscription using a device code. Once you are connected to your workspace in Azure cloud, you can start experimenting.

The workspace is an Azure resource that contains your experiments, models, deployments and remote compute resources, and the experiment is a container that you use to track and organize your runs.

In [5]:
!az account show

{
  "environmentName": "AzureCloud",
  "id": "1baa1f3f-9b61-4f60-9da8-3a76ba416716",
  "isDefault": true,
  "name": "Cosmos_OPG_Office365_BPOS_100119",
  "state": "Enabled",
  "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  "user": {
    "name": "andyxu@microsoft.com",
    "type": "user"
  }
}


In [6]:
from azureml.core import Workspace, Experiment, Run
import math, random, pickle

In [7]:
ws = Workspace.from_config()
print(ws)

Found the config file in: /home/nbuser/library/config.json


In [11]:
print(ws.subscription_id)
print(ws.resource_group)

ws_list = Workspace.list(ws.subscription_id)
print(ws_list)

bc69d98c-7d2b-4542-88a4-f86eb4aea4a5
USEastGroup_AX
{'aidemosdevaml': [<azureml.core.workspace.Workspace object at 0x7fd4d0903eb8>], 'aidemos': [<azureml.core.workspace.Workspace object at 0x7fd4d0851940>], 'jehyrkas-test': [<azureml.core.workspace.Workspace object at 0x7fd4d08519b0>], 'mtw1920ccc74a8a45879': [<azureml.core.workspace.Workspace object at 0x7fd4d0851198>], 'mtw1b1a1a9f3c354180b': [<azureml.core.workspace.Workspace object at 0x7fd4d0851208>], 'aml_hack_ps': [<azureml.core.workspace.Workspace object at 0x7fd4d0851908>], 'automl_scikit': [<azureml.core.workspace.Workspace object at 0x7fd4d0851da0>], 'bugbash_ws1204': [<azureml.core.workspace.Workspace object at 0x7fd4d0851b38>], 'hyperdrive_scikit': [<azureml.core.workspace.Workspace object at 0x7fd4d0851358>], 'pyspark_ws1304': [<azureml.core.workspace.Workspace object at 0x7fd4d0851080>], 'GalleryTestWorkSpace': [<azureml.core.workspace.Workspace object at 0x7fd4d08510b8>], 'AutoML': [<azureml.core.workspace.Workspace obj

In [5]:
experiment = Experiment(workspace_object=ws, name = "my-first-experiment")

Next, let's start an experiment run. We use Monte Carlo simulation to estimate pi as a simple example. While the experiment is running, metrics about the accuracy of the estimate are logged into run history.

Let's also save the value as a file into run history. In the next notebook we'll use this file to create a web service that computes an area of a circle using our estimate.

In [6]:
run = experiment.start_logging()

pi_counter = 0
n_iter = 100000

# Log total number of iterations
run.log("Number of iterations",n_iter)

for i in range(1,n_iter):
    # Monte Carlo step to update estimate
    x = random.random()
    y = random.random()
    if x*x + y*y < 1.0:
        pi_counter += 1
    pi_estimate = 4.0*pi_counter / i
    
    # Log convergence every 10000 iterations
    if i%10000==0:
        error = math.pi-pi_estimate
        run.log("Pi estimate",pi_estimate)
        run.log("Error",error)

# Log final results
run.log("Final estimate: ",pi_estimate)
run.log("Final error: ",math.pi-pi_estimate)

# Write file containing pi value into run history
with open("pi_estimate.txt","wb") as f:
    pickle.dump(str(pi_estimate),f)
run.upload_file(name = 'outputs/pi_estimate.txt', path_or_stream = './pi_estimate.txt')

# Complete tracking and get link to details
run.complete()

Once the run has completed, you can view a detailed report of the run from Azure Portal by simply calling "run" and following the link.

In [7]:
run

Experiment,Id,Type,Status,Details Page
my-first-experiment,4f4a5891-04e6-459b-87d8-b4f69547ba4e,,Completed,Link to Azure Portal


Next, learn how to deploy a web service that computes the area of circle using your estimate using following Notebook:

[Deploy web service](02.deploy-web-service.ipynb)